### Install dependencies

This notebook requires a dependency which can be installed with the following command `pip install open-bus-stride-client`.

You can also launch it online at [this URL](https://mybinder.org/v2/gh/hasadna/open-bus-stride-client/main?labpath=notebooks%2Fload%20gtfs%20timetable%20to%20pandas%20dataframe.ipynb), when launching online the dependencies are already installed.


In [1]:
# !pip install open-bus-stride-client

In [2]:
import pandas as pd
import datetime
from dateutil import tz

pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 1000

import stride

### Find a route to investigate

First, let's find a route (`line_ref`) to investigate. As each line (`route_short_name`) could have few routes (mainly one for each driving direction), we first take a look on the `gtfs_routes` for picking our route.  

Let's look for line number `15` (Egged, Jerusalem).

In [3]:
pd.DataFrame(stride.get('/gtfs_routes/list', {'route_short_name':15,
                                                            'agency_name': 'אגד',
                                                            'route_long_name_contains': 'ירושלים',
                                              'date_from': '2022-06-18',
                                              'date_to':  '2022-06-18'}))

,id,date,line_ref,operator_ref,route_short_name,route_long_name,route_mkt,route_direction,route_alternative,agency_name,route_type
0,702184,2022-06-18,12405,3,15,תחנה תפעולית/ביטוח לאומי-ירושלים<->האומן/ברעם-ירושלים-1#,59015,1,#,אגד,3
1,702185,2022-06-18,12406,3,15,האומן/ברעם-ירושלים<->שדרות יצחק רבין/נתנאל לורך-ירושלים-2#,59015,2,#,אגד,3


This line as 2 routes (`line_ref`), let's pick `14206` route and get its timetable (note GTFS is the planned timetable, not the actual).

### Get GTFS Timetable (Documentation: https://open-bus-stride-api.hasadna.org.il/docs#/)

We use the stride iterate method to efficiently iterate over a possibly long list of results.

Behind the scenes it uses the offset/limit parameters so you don't have to worry about it.

We pass on the iterator directly on to Pandas to create a DataFrame.

In [4]:
gtfs_timetable_15 = pd.DataFrame(stride.iterate('/route_timetable/list', {
    'line_refs': '12406',
    'planned_start_time_date_from': '2022-06-08T08:00:00+03:00',
    'planned_start_time_date_to': '2022-06-08T08:00:00+03:00',
    'order_by': 'gtfs_line_start_time, planned_arrival_time'
}, limit=10000))

gtfs_timetable_15.shape

(26, 9)

Note that in this example the `planned_start_time_date_from` and the `planned_start_time_date_to` are both equal to a spesific ride start time. You can also pass a time window of few minutes/hours to (potentially) get more than one ride timetable for the same route (`line_ref`).

The date columns are on UTC timezone, let's localize the dates to Israel timezone.

In [5]:
def localize_dates(data, dt_columns = []):
    data = data.copy()
    
    for c in dt_columns:
        data[c] = pd.to_datetime(data[c]).dt.tz_convert('Israel')
    
    return data

In [6]:
dt_columns = ['gtfs_line_start_time','planned_arrival_time']

gtfs_timetable_15 = localize_dates(gtfs_timetable_15, dt_columns)

In [7]:
gtfs_timetable_15[['gtfs_line_ref','gtfs_line_start_time', 'city','name','planned_arrival_time', 'lon','lat']]

,gtfs_line_ref,gtfs_line_start_time,city,name,planned_arrival_time,lon,lat
0,12406,2022-06-08 08:00:00+03:00,ירושלים,האומן/ברעם,2022-06-08 08:00:00+03:00,35.213151,31.746651
1,12406,2022-06-08 08:00:00+03:00,ירושלים,האומן/הסדנא,2022-06-08 08:00:59+03:00,35.211292,31.748705
2,12406,2022-06-08 08:00:00+03:00,ירושלים,הסדנא/פייר קניג,2022-06-08 08:01:59+03:00,35.213997,31.750180
3,12406,2022-06-08 08:00:00+03:00,ירושלים,יד חרוצים/התנופה,2022-06-08 08:02:25+03:00,35.215654,31.750171
4,12406,2022-06-08 08:00:00+03:00,ירושלים,יד חרוצים/פועלי צדק,2022-06-08 08:03:23+03:00,35.216619,31.752369
5,12406,2022-06-08 08:00:00+03:00,ירושלים,פועלי צדק,2022-06-08 08:04:05+03:00,35.215168,31.753290
6,12406,2022-06-08 08:00:00+03:00,ירושלים,קניון הדר/פייר קניג,2022-06-08 08:04:59+03:00,35.214349,31.754216
7,12406,2022-06-08 08:00:00+03:00,ירושלים,פייר קניג/חלץ,2022-06-08 08:05:59+03:00,35.214837,31.756866
8,12406,2022-06-08 08:00:00+03:00,ירושלים,צומת אורנים/פייר קניג,2022-06-08 08:06:34+03:00,35.215225,31.758569
9,12406,2022-06-08 08:00:00+03:00,ירושלים,בן זכאי/פייר קניג,2022-06-08 08:07:28+03:00,35.213552,31.759129
